# Training MLP model on insurance claims data

In [9]:
from dydx.layers import Linear, Loss
from dydx.model import Model
from dydx.dataset import DataLoader, Dataset

l1 = Linear( random=True, dims=(8, 64))
l2 = Linear( random=True,dims=(64, 64))
l3 = Linear( random=True,dims=(64, 16))
l4 = Linear( activation=False, random=True, dims=(16,1)
)

		
names = ["l1","l2","l3","l4"]		
layers =[l1,l2,l3,l4]
ls = dict(zip(names,layers))

model = Model(ls) 
loss = Loss()
EPOCHS= 100

ds = Dataset(testing=True) # takes 160 examples to overfit model too for testing purposes
splits = ['train','val','test']
print(f"dataset sizes for {', '.join(splits)} are {[ds.__len__(split) for split in splits]} respectively.")
dst = ds.train
dsv = ds.val
dss = ds.test
# iterators 
dltrain  = DataLoader(dst,16)()
dlval = DataLoader(dsv,16)()
dltest = DataLoader(dss,16)()

for epoch in range(EPOCHS):
	print("Training".center(70,"#"))
	for (idx,xy) in enumerate( dltrain):
		x,y = xy 
		model.zero_grad()	
		y_hat =model(x) 
		
		acc, avg_loss = loss(y,y_hat)
		avg_loss.backward()
		
		print(f'Epoch {epoch}:{idx} Loss:{avg_loss.data} train accuracy:{acc*100}%')
			
		lr = 0.0005 
		for p in model.parameters():
			print('p.grad', p.grad)				
			p.data = p.data -  lr * p.grad
		
		
	print("Validation".center(70,"#"))
	for (idx,xy) in enumerate( dlval):
		x,y = xy 
#			for layer in model.layers:
#				x = layer(x)
		y_hat = model(x)
		acc, avg_loss= loss(y,y_hat)
		print(f'Epoch {epoch}:{idx} Loss:{avg_loss.data} val accuracy:{acc*100}%')	

# train()

[([28.0, 5, 1, 28.13, 70, 0, 112.5, 10], [1.0]), ([37.0, 13, 1, 12.95, 95, 0, 37.0, 23], [0.0]), ([27.0, 3, 0, 0.0, 89, 0, 13.0, 20], [0.0]), ([36.0, 3, 0, 0.0, 72, 0, 34.0, 20], [0.0]), ([26.0, 0, 0, 23.76, 46, 0, 39.6, 4], [0.0])]
dataset sizes for train, val, test are [112, 32, 16] respectively.
###############################Training###############################
indide loss y_hat
 [[(data:0.42464995580300474, grad:0)], [(data:0.4008310998967705, grad:0)], [(data:0.4159988419703904, grad:0)], [(data:0.40693683530329977, grad:0)], [(data:0.42687296760653837, grad:0)], [(data:0.3996268285710462, grad:0)], [(data:0.4282007457800642, grad:0)], [(data:0.41288436893429176, grad:0)], [(data:0.40652331832307276, grad:0)], [(data:0.40628905080313027, grad:0)]]
 inside loss y
 [(data:1.0, grad:0), (data:0.0, grad:0), (data:0.0, grad:0), (data:1.0, grad:0), (data:1.0, grad:0), (data:0.0, grad:0), (data:1.0, grad:0), (data:1.0, grad:0), (data:1.0, grad:0), (data:0.0, grad:0)]
Epoch 0:0 Loss:1

ZeroDivisionError: 0.0 cannot be raised to a negative power